# 이미지에서 상품 속성 추출

이 노트북에서는 GPT-4o Vision을 활용하여 상품 이미지에서 자동으로 속성을 추출합니다.

### 주요 기능
- 🖼️ **이미지 분석**: GPT-4o Vision으로 상품 이미지 분석
- 📋 **속성 구조화**: OpenAI Structured Output으로 JSON 포맷 보장

## 0. 필요한 라이브러리 설치

In [ ]:
##############################################
# 0. 라이브러리 설치 (최초 1회 실행)
##############################################
%pip install --quiet python-dotenv openai pillow requests azure-search-documents azure-identity pydantic

## 1. 환경 변수 설정

In [ ]:
import os
from dotenv import load_dotenv

##############################################
# 1. 환경 변수 로드 및 확인
##############################################
load_dotenv()

# AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION", "2024-10-21")
AZURE_OPENAI_ENDPOINT  = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_VISION_DEPLOYMENT  = os.getenv("AZURE_OPENAI_VISION_DEPLOYMENT")

AZURE_SEARCH_SERVICE_ENDPOINT = os.getenv("AZURE_SEARCH_SERVICE_ENDPOINT")
AZURE_SEARCH_INDEX_NAME = os.getenv("AZURE_SEARCH_INDEX_NAME")

print(f"OpenAI Api Version: {AZURE_OPENAI_API_VERSION}")
print(f"OpenAI Endpoint: {AZURE_OPENAI_ENDPOINT}")
print(f"OpenAI Vision Deployment: {AZURE_OPENAI_VISION_DEPLOYMENT}")
print(f"Search Endpoint: {AZURE_SEARCH_SERVICE_ENDPOINT}")
print(f"Search Index: {AZURE_SEARCH_INDEX_NAME}")

## 2. 초기화 및 환경 설정

In [ ]:
import os
import base64
import json
from openai import AzureOpenAI
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

import requests

# 환경 변수 로드
load_dotenv()

# Azure OpenAI 서비스 인증 설정 (임베딩용)
credential = DefaultAzureCredential()
token_provider = get_bearer_token_provider(
    credential,
    "https://cognitiveservices.azure.com/.default"
)

# Azure OpenAI 클라이언트 초기화
client = AzureOpenAI(
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    azure_ad_token_provider=token_provider
)

## 3. 기본 이미지 분석 테스트

먼저 간단한 이미지 분석을 테스트해봅시다.

In [ ]:
# 간단한 설명 요청
simple_prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
sample_image_url = "https://images.unsplash.com/photo-1525966222134-fcfa99b8ae77?w=800&q=80"

response = client.chat.completions.create(
    model=AZURE_OPENAI_VISION_DEPLOYMENT,
    messages=[
        {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": simple_prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": sample_image_url},
                }
            ]
        }
    ],
    max_tokens=2048
)

result = response.choices[0].message.content

print("🖼️ 이미지 분석 결과:")
print("="*80)
print(result)
print("="*80)

## 4. 구조화된 상품 속성 추출

이제 실제 이커머스에서 필요한 구조화된 데이터를 추출해봅시다.

In [ ]:
# 간단한 설명 요청
simple_prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
sample_image_url = "https://images.unsplash.com/photo-1525966222134-fcfa99b8ae77?w=800&q=80"

# JSON Schema 정의
schema = {
    "type": "object",
    "properties": {
        "imageCaption": {"type": "string", "description": "한국어 캡션"},
        "imageDescription": {"type": "string", "description": "한국어 자세한 설명"},
        "imageTags": {
            "type": "array",
            "items": {"type": "string"},
            "description": "핵심 태그 목록"
        }
    },
    "required": ["imageCaption", "imageDescription", "imageTags"],
    "additionalProperties": False
}
    
response = client.chat.completions.create(
    model=AZURE_OPENAI_VISION_DEPLOYMENT,
    messages=[
        {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": simple_prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": sample_image_url},
                }
            ]
        }
    ],
    # JSON Schema 포맷 지시
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "ImageCaptionSchema", "schema": schema}
    },
    max_tokens=2048
)

result = response.choices[0].message.content

print("🖼️ 이미지 분석 구조화된 결과:")
print("="*80)
result_json = json.loads(result)
print(json.dumps(result_json, indent=2, ensure_ascii=False))
print("="*80)

In [ ]:
import os
from pydantic import BaseModel

# 응답 포맷 지정
class ProductInfo(BaseModel):
    imageCaption: str
    imageDescription: str
    imageTags: list[str]

# 간단한 설명 요청
simple_prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
sample_image_url = "https://images.unsplash.com/photo-1525966222134-fcfa99b8ae77?w=800&q=80"
    
completion = client.chat.completions.parse(
    model=AZURE_OPENAI_VISION_DEPLOYMENT,
    messages=[
        {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
        {
            "role": "user",
            "content": [
                {"type": "text", "text": simple_prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": sample_image_url},
                }
            ]
        }
    ],
    # JSON Schema 포맷 지시
    response_format=ProductInfo,
    max_tokens=2048
)

event = completion.choices[0].message.parsed

print("🖼️ 이미지 분석 구조화된 결과:")
print("="*80)
print(event.imageCaption)
print(event.imageDescription)
print(event.imageTags)
print("="*80)

## 5. 기존 제품 정보를 확장


In [ ]:
##############################################
# 5-1. 기존 제품 데이터 로드
##############################################
import json
from pathlib import Path

# 데이터 파일 경로
data_path = Path("data/sample_products.json")

# JSON 파일 읽기
with open(data_path, "r", encoding="utf-8") as f:
    products = json.load(f)

print(f"✅ 총 {len(products)}개의 제품 데이터를 로드했습니다.")
print(f"\n첫 번째 제품 예시:")
print(json.dumps(products[0], indent=2, ensure_ascii=False))

In [ ]:
##############################################
# 5-2. 이미지 분석 함수 정의
##############################################
from pydantic import BaseModel

class ProductInfo(BaseModel):
    imageCaption: str
    imageDescription: str
    imageTags: list[str]

def analyze_product_image(image_url: str, retry_count: int = 3) -> ProductInfo:
    """
    제품 이미지를 분석하여 캡션, 설명, 태그를 추출합니다.
    
    Args:
        image_url: 분석할 이미지 URL
        retry_count: 실패 시 재시도 횟수
        
    Returns:
        ProductInfo: 이미지 분석 결과
    """
    prompt = "이 이미지에 있는 상품에 대해서 고객이 검색해보기 편하도록 자세하게 한글로 설명해주세요."
    
    for attempt in range(retry_count):
        try:
            completion = client.chat.completions.parse(
                model=AZURE_OPENAI_VISION_DEPLOYMENT,
                messages=[
                    {"role": "system", "content": "상품에 대한 정보를 한글로 추출합니다."},
                    {
                        "role": "user",
                        "content": [
                            {"type": "text", "text": prompt},
                            {
                                "type": "image_url",
                                "image_url": {"url": image_url},
                            }
                        ]
                    }
                ],
                response_format=ProductInfo,
                max_tokens=2048
            )
            
            return completion.choices[0].message.parsed
            
        except Exception as e:
            if attempt < retry_count - 1:
                print(f"⚠️ 재시도 중... ({attempt + 1}/{retry_count})")
                continue
            else:
                print(f"❌ 이미지 분석 실패: {e}")
                raise
                
print("✅ 이미지 분석 함수가 정의되었습니다.")

In [ ]:
##############################################
# 5-3. 모든 제품 이미지 분석 및 데이터 업데이트
##############################################
from time import sleep
from tqdm.notebook import tqdm

# 업데이트된 제품 데이터를 저장할 리스트
augmented_products = []

print(f"🚀 {len(products)}개의 제품 이미지를 분석합니다...\n")

# 각 제품의 이미지 분석
for idx, product in enumerate(tqdm(products, desc="제품 이미지 분석")):
    try:
        # 이미지 URL 확인
        image_url = product.get("imageUrl")
        
        if not image_url:
            print(f"⚠️ 제품 ID {product.get('id')}: 이미지 URL이 없습니다. 건너뜁니다.")
            augmented_products.append(product)
            continue
        
        # 이미지 분석
        print(f"\n📸 [{idx+1}/{len(products)}] {product.get('name', 'Unknown')} 분석 중...")
        result = analyze_product_image(image_url)
        
        # 기존 제품 데이터 복사 및 업데이트
        updated_product = product.copy()
        updated_product["imageCaption"] = result.imageCaption
        updated_product["imageDescription"] = result.imageDescription
        updated_product["imageTags"] = result.imageTags
        
        augmented_products.append(updated_product)
        
        print(f"   ✅ 캡션: {result.imageCaption[:50]}...")
        print(f"   ✅ 설명: {result.imageDescription[:50]}...")
        print(f"   ✅ 태그: {', '.join(result.imageTags[:5])}")
        
        # API Rate Limit 고려하여 짧은 대기
        sleep(0.5)
        
    except Exception as e:
        print(f"❌ 제품 ID {product.get('id')} 처리 중 오류: {e}")
        # 오류 발생 시 원본 데이터 유지
        augmented_products.append(product)
        continue

print(f"\n✅ 총 {len(augmented_products)}개의 제품 처리 완료!")

In [ ]:
##############################################
# 5-4. 업데이트된 데이터를 파일로 저장
##############################################

# 저장할 파일 경로
output_path = Path("data/sample_products_augmented.json")

# JSON 파일로 저장
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(augmented_products, f, ensure_ascii=False, indent=2)

print(f"✅ 업데이트된 제품 데이터가 저장되었습니다: {output_path}")
print(f"📊 총 {len(augmented_products)}개의 제품")

# 결과 샘플 출력
print(f"\n📋 업데이트된 첫 번째 제품 예시:")
print("="*80)
print(json.dumps(augmented_products[0], indent=2, ensure_ascii=False))
print("="*80)